In [ ]:
# Imports

import os
import joblib
import pandas as pd

from src.modsec import init_modsec
from src.utils import load_data_label_vector, get_most_recent_data_path
from src.model import train_model, test_evasion
from config import BaseConfig, HalfConfig, PaperConfig

In [ ]:
# Setup Variables

Config = PaperConfig()
data_path = get_most_recent_data_path()

print(f"Using data from {data_path}")

In [ ]:
# Load Data

train = load_data_label_vector(f"{data_path}/train.csv")
test = load_data_label_vector(f"{data_path}/test.csv")
train_adv = load_data_label_vector(f"{data_path}/train_adv.csv")
test_adv = load_data_label_vector(f"{data_path}/test_adv.csv")

# print shape
print(f"Train: {train.shape}, Test: {test.shape}")
print(f"Train Adv: {train_adv.shape}, Test Adv: {test_adv.shape}")

In [ ]:
# Load/Train Models

model_trained = joblib.load(f"{data_path}/model.joblib")
threshold = float(open(f"{data_path}/threshold.txt", "r").read())

# # remove the min_confidence column in adv sets
# train_adv = train_adv.drop(columns=["min_confidence"])
# test_adv = test_adv.drop(columns=["min_confidence"])

model_adv_trained, threshold_adv = train_model(
    train=pd.concat([train, train_adv]).sample(frac=1).reset_index(drop=True),
    test=pd.concat([test, test_adv]).sample(frac=1).reset_index(drop=True),
    model=Config.MODEL_ADV,
    desired_fpr=Config.DESIRED_FPR,
)

In [ ]:
# Test Evasion (base model without adversarial training)

test_evasion(
    payload='SELECT SLEEP(5)#";',
    threshold=threshold,
    model=model_trained,
    engine_eval_settings={
        "max_rounds": 2000,
        "round_size": 30,
        "timeout": 120,
        "threshold": 0.0,
    },
    modsec=init_modsec(),
    rule_ids=Config.RULE_IDS,
    paranoia_level=Config.PARANOIA_LEVEL,
)

In [ ]:
# Test Evasion (adversarially trained model)

test_evasion(
    payload='SELECT SLEEP(5)#";',
    threshold=threshold,
    model=model_adv_trained,
    engine_eval_settings={
        "max_rounds": 2000,
        "round_size": 30,
        "timeout": 120,
        "threshold": 0.0,
    },
    modsec=init_modsec(),
    rule_ids=Config.RULE_IDS,
    paranoia_level=Config.PARANOIA_LEVEL,
)